# **Administração de Sistemas - Laboratório 2**

## **ESTE FICHEIRO É A RESOLUÇÃO DO LABORATÓRIO DE 2020-2021**

## **Instalação de Máquinas**

* **Servidor** - Versão minimal CentOs
* **Cliente** - Windows

## **Configurações da VM**

* Disco -> 10gb
* Memória RAM -> 1gb
* Mudar placa de rede para **"Bridged Adapter"**

## **Configurações do CentOS**

* Mudar linguagem
* Mudar esquema do teclado
* Desativar o Kdump
* Na parte de network
    * Mudar o hostname
    * Ativar a network
* Desativar a *Security Policy*
* Partições
    * *I will configure partitioning*
    * *Standard Partition*
        * **"/boot"** -> 250m
        * **"swap"** -> 2g
        * **"/"** -> deixar em branco
        * Mudar **"/boot"** e **"/"** para **ext4** 
* Colocar password de root

## **Primeiros Passos**

### **Servidor e Cliente**

#### **Desativar a firewall**

* **Comandos**
    * systemctl stop firewalld
    * systemctl disable firewalld
    * vi /etc/selinux/config
        * Remover a parte onde diz **"SELINUX=enforcing"** para **"SELINUX=disabled"**
    * reboot   

* **Instalar Serviços**
    * **Servidor**
        * **Comandos**
            * yum install net-tools -y
            * yum install bind* -y
            * yum install httpd -y
            * yum install mdadm -y
    * **Client**
        * **Comandos**
            * yum install net-tools -y


## **Criação de users e grupos**

#### **Criação de user**

* **Comandos**
    * useradd nomeuser
    * passwd nomeuser (definir pass para o user)

##### **Criação de user num determinado diretório**

* **Comandos**
    * criar o diretório primeiro
    * useradd -m -d /storage/nomeuser nomeuser (/storage/nomeuser é o caminho pretendido)
    * passwd nomeuser (definir pass para o user)

#### **Criação de grupos**

* **Comandos**
    * Criar um grupo
        * groupadd nomegrupo
    * Adicionar users ao grupo
        * gpasswd -a nomeuser nomegrupo
    * Definir administrador do grupo
        * gpasswd -A nomeuser nomegrupo
    * Remover users do grupo (apenas admin do grupo)
        * gpasswd -d nomeuser nomegrupo
    * Diretoria comum para o grupo
        * mkdir /home/nomepastagrupo
        * chgrp nomegrupo /home/nomepastagrupo
        * chmod g+ws /home/nomepastagrupo

## **DNS SERVER**

#### **Servidor**

* vi /etc/named.conf
    * Acrescentar a opção "any;" aos parâmetros *listen-on port 53* e *allow-query*
    * Adicionar as seguintes configurações das zonas *master* e *reverse* para os domínios pedidos no enunciado:
        * zone "gules.org" IN { <br />
            type master; <br />
            file "/var/named/gules.org.hosts"; <br />
        }; 
        * zone "300emfrente.eu" IN { <br />
            type master; <br />
            file "/var/named/300emfrente.eu.hosts"; <br />
        }
        * zone "then.com" IN { <br />
            type master; <br />
            file "/var/named/then.com.hosts"; <br />
        }
        * zone "22.200.191.in-addr.arpa" IN { <br />
            type master; <br />
            file "/var/named/22.200.191.in-addr.arpa.hosts"; <br />
        }
        * zone "45.147.92.in-addr.arpa" IN { <br />
            type master; <br />
            file "/var/named/45.147.92.in-addr.arpa.hosts"; <br />
        }
        * zone "22.168.194.in-addr.arpa" IN { <br />
            type master; <br />
            file "/var/named/22.168.194.in-addr.arpa.hosts"; <br />
        }
* De seguida criar os ficheiros *hosts*
* vi /var/named/gules.org.hosts
    * Colocar no ficheiro a seguinte configuração:


* vi /var/named/300emfrente.eu.hosts

* vi /var/named/then.com.hosts

* vi /var/named/22.200.191.in-addr.arpa.hosts

* vi /var/named/45.147.92.in-addr.arpa.hosts

* vi /var/named/22.168.194.in-addr.arpa.hosts

* systemctl enable named
* systemctl start named

### **TESTE NA MÁQUINA CLIENTE**

Mudar DNS da placa de rede para o IP do servidor

## **HTTP SERVER**

#### **Servidor**

* systemctl start httpd
* systemctl enable httpd
* vi /etc/httpd/conf/httpd.conf
    * Na secção “\<Directory "/">” do ficheiro, alterar a opção “AllowOverride
        none” para “AllowOverride AuthConfig”.
    * Na secção “\<IfModule dir_module>” alterar “DirectoryIndex index.html”
        para “DirectoryIndex inicio.html”.
* vi /etc/httpd/conf.d/userdir.conf
    * No ficheiro comentar a opção “UserDir disabled” e colocar “UserDir
        homepage” e na secção inferior do ficheiro alterar “\<Directory
        “/home/*/public_html”>” para “\<Directory “/home/\*/homepage”>”.
* useradd asuser1
* passwd asuser1 (definir pass do user)
* usermod -a -G users asuser1
* mkdir /home/asuser1/homepage
* vi /home/asuser1/homepage/inicio.html
    * Escrever no ficheiro "asuser1"
* cd /home/asuser1
* chown asuser1:asuser1 homepage -R
* chmod 755 /home/asuser1 -R

* useradd asuser2
* passwd asuser2 (definir pass do user)
* usermod -a -G users asuser2
* mkdir /home/asuser2/homepage
* vi /home/asuser2/homepage/inicio.html
    * Escrever no ficheiro "asuser2"
* cd /home/asuser2
* chown asuser2:asuser2 homepage -R
* chmod 755 /home/asuser2 -R
* systemctl restart httpd

## **APACHE**

* cd /home/asuser1/homepage
* mkdir private
* vi .htaccess
    * Colocar no ficheiro as seguintes instruções:
        *   AuthName "Diretorio Privado – asuser1" <br />
            AuthType Basic <br />
            AuthUserFile /home/asuser1/.user_passwd <br />
            require valid-user <br />
* cd /home/asuser1
* htpasswd -c .user_passwd private (definir pass)
* htpasswd .user_passwd privado (definir pass)

* cd /home/asuser2/homepage
* mkdir private
* vi .htaccess
    * Colocar no ficheiro as seguintes instruções:
        *   AuthName "Diretorio Privado – asuser2" <br />
            AuthType Basic <br />
            AuthUserFile /home/asuser1/.user_passwd <br />
            require valid-user <br />
* cd /home/asuser2
* htpasswd -c .user_passwd rdis (definir pass)


## **VIRTUALHOSTS**
## **IP É O DO SERVIDOR!!!!**

* vi /etc/named.conf
    * Acrescentar a opção "any; " aos parâmetros listen-on port 53 e allowquery
    * Adicionar as seguintes configurações das zonas master e reverse   para os domínios pedidos no enunciados :
        * zone "allow.org" IN { <br />
            type master; <br />
            file "/var/named/allow.org.hosts"; <br />
        }; <br />
        * zone "circle360.pt" IN { <br />
            type master; <br />
            file "/var/named/circle360.pt.hosts"; <br />
        }; <br />
        * zone "festas.pt" IN { <br />
            type master; <br />
            file "/var/named/festas.pt.hosts"; <br />
        };
* vi /var/named/allow.org.hosts
    * Colocar no ficheiro a seguinte configuração:

## **IP É O DO SERVIDOR!!!!**
* vi /var/named/circle360.pt.hosts
    * Colocar no ficheiro a seguinte configuração:

## **IP É O DO SERVIDOR!!!!**
* vi /var/named/festas.pt.hosts
    * Colocar no ficheiro a seguinte configuração:

## **IP É O DO SERVIDOR!!!!**
* vi /etc/httpd/conf/httpd.conf
    * Acrescentar as instruções “Listen 25000” e “Listen 28000” para o
        servidor conseguir responder a pedidos nas portas 25000 e 28000,
        respetivamente;
    * De modo a alojar as páginas dos 3 domínios nas portas 25000 e     28000, é necessário acrescentar a seguinte configuração dos VirtualHosts :
        * NameVirtualHost 192.168.1.101:25000 <br />
          NameVirtualHost 192.168.1.101:28000 <br />

          \<VirtualHost 192.168.1.101:25000 192.168.1.101:28000> <br />
          DocumentRoot "/dominios/allow.org/" <br />
          ServerName www.allow.org <br />
          ServerAlias allow.org <br />
          \<Directory "/dominios/allow.org"> <br />
          Options Indexes FollowSymLinks <br />
          AllowOverride All <br />
          Order allow,deny <br />
          Allow from all <br />
          Require method GET POST OPTIONS <br />
          \</Directory> <br />
          \</VirtualHost> <br />

          \<VirtualHost 192.168.1.101:25000 192.168.1.101:28000> <br />
          DocumentRoot "/dominios/circle360.pt/" <br />
          ServerName www.circle360.pt <br />
          ServerAlias circle360.tp <br />
          \<Directory "/dominios/circle360.pt"> <br />
          Options Indexes FollowSymLinks <br />
          AllowOverride All <br />
          Order allow,deny <br />
          Allow from all <br />
          Require method GET POST OPTIONS <br />
          \</Directory> <br />
          \</VirtualHost> <br />

          \<VirtualHost 192.168.1.101:25000 192.168.1.101:28000> <br />
          DocumentRoot "/dominios/festas.pt/" <br />
          ServerName www.festas.pt <br />
          ServerAlias festas.pt <br />
          \<Directory "/dominios/festas.pt"> <br />
          Options Indexes FollowSymLinks <br />
          AllowOverride All <br />
          Order allow,deny <br />
          Allow from all <br />
          Require method GET POST OPTIONS <br />
          \</Directory> <br />
          \</VirtualHost> <br />

* mkdir /dominios
* mkdir /dominios/allow.org
* vi /dominios/allow.org/inicio.html
    * Escrever no ficheiro "allow.org"
* mkdir /dominios/circle360.pt
* vi /dominios/circle360.pt/inicio.html
    * Escrever no ficheiro "circle360.pt"
* mkdir /dominios/festas.pt
* vi /dominios/festas.pt/inicio.html
    * Escrever no ficheiro "festas.pt"
* chmod 755 /dominios -R
* systemctl restart named
* systemctl restart httpd
        

## **RAID**

* Desligar a máquina
* Criar os discos no virtual box
* mdadm --create --verbose /dev/md0 --level=1 --raid-devices=2 /dev/sdb /dev/sdc --spare-devices=1 /dev/sdd
* mkfs.ext4 /dev/md0
* mkdir /dataraid1
* mount /dev/md0 /dataraid1


#### **Comandos úteis**

* mdadm --detail /dev/md0 -> Ver o estado da RAID
* mdadm --manage /dev/md0 -> Danificar o disco
* mdadm --manage /dev/md0 -> Remover o disco da RAID
* mdadm --add /dev/md0 /dev/sde -> Adicionar um novo disco á RAID
